In [10]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
import json

load_dotenv()
YELP_API_KEY = os.getenv('YELP_API_KEY')

In [11]:
url = 'https://api.yelp.com/v3/categories'

headers = {'Authorization': f'Bearer {YELP_API_KEY}'}

response = requests.get(url, headers=headers)
parsed = json.loads(response.text)
print(json.dumps(parsed, indent=4))

{
    "categories": [
        {
            "alias": "3dprinting",
            "title": "3D Printing",
            "parent_aliases": [
                "localservices"
            ],
            "country_whitelist": [],
            "country_blacklist": []
        },
        {
            "alias": "abruzzese",
            "title": "Abruzzese",
            "parent_aliases": [
                "italian"
            ],
            "country_whitelist": [
                "IT"
            ],
            "country_blacklist": []
        },
        {
            "alias": "absinthebars",
            "title": "Absinthe Bars",
            "parent_aliases": [
                "bars"
            ],
            "country_whitelist": [
                "CZ"
            ],
            "country_blacklist": []
        },
        {
            "alias": "acaibowls",
            "title": "Acai Bowls",
            "parent_aliases": [
                "food"
            ],
            "country_whitelist": [],
      

In [17]:
categories_list = set()

for category in parsed['categories']:
    try:
        categories_list.add(category['parent_aliases'][0])
    except:
        continue
print(list(categories_list))

['beautysvc', 'artsandcrafts', 'restaurants', 'tcm', 'realestate', 'adultentertainment', 'physicians', 'psychic_astrology', 'nightlife', 'social_clubs', 'african', 'brazilian', 'bars', 'donburi', 'german', 'photographers', 'health', 'media', 'autorepair', 'caribbean', 'autopartssupplies', 'laundryservices', 'museums', 'utilities', 'auto', 'dentists', 'homeservices', 'arts', 'malaysian', 'hair', 'publicservicesgovt', 'food', 'farms', 'petstore', 'portuguese', 'transport', 'polish', 'tours', 'tastingclasses', 'diving', 'japanese', 'hairremoval', 'travelservices', 'landscaping', 'crisispregnancycenters', 'cafes', 'festivals', 'skincare', 'latin', 'funeralservices', 'active', 'jpsweets', 'gardening', 'dentalhygienists', 'opthamalogists', 'mexican', 'estateplanning', 'legalservices', 'hotelstravel', 'education', 'insurance', 'fashion', 'wineries', 'realestateagents', 'movietheaters', 'arabian', 'lawyers', 'artclasses', 'opticians', 'gourmet', 'junkremovalandhauling', 'realestatesvcs', 'pet_

In [22]:

categories_query = ','.join(categories_list)
# categories_list = list(categories_list).join(',')
print(categories_query)
    

beautysvc,artsandcrafts,restaurants,tcm,realestate,adultentertainment,physicians,psychic_astrology,nightlife,social_clubs,african,brazilian,bars,donburi,german,photographers,health,media,autorepair,caribbean,autopartssupplies,laundryservices,museums,utilities,auto,dentists,homeservices,arts,malaysian,hair,publicservicesgovt,food,farms,petstore,portuguese,transport,polish,tours,tastingclasses,diving,japanese,hairremoval,travelservices,landscaping,crisispregnancycenters,cafes,festivals,skincare,latin,funeralservices,active,jpsweets,gardening,dentalhygienists,opthamalogists,mexican,estateplanning,legalservices,hotelstravel,education,insurance,fashion,wineries,realestateagents,movietheaters,arabian,lawyers,artclasses,opticians,gourmet,junkremovalandhauling,realestatesvcs,pet_sitting,sportswear,italian,chinese,sportgoods,airports,religiousorgs,turkish,cannabis_clinics,breakfast_brunch,partyequipmentrentals,plumbing,c_and_mh,parks,diagnosticservices,financialservices,localflavor,medicalspa,z

In [25]:
params = {
    "latitude": "39.767347",
    "longitude": "-104.949932",
    "radius": "956",
    "limit": "50",
    "categories": categories_query,
    "offset": 0
}
headers = {'Authorization': f'Bearer {YELP_API_KEY}'}
url = 'https://api.yelp.com/v3/businesses/search'

businesses_list = []
response = requests.get(url, headers=headers, params=params)
parsed = json.loads(response.text)
businesses_list.append(parsed)

while parsed['businesses']:
    response = requests.get(url, headers=headers, params=params)
    parsed = json.loads(response.text)
    businesses_list.append(parsed)
    params['offset'] += 50

In [42]:
print(json.dumps(businesses_list[0], indent=4))

{
    "businesses": [
        {
            "id": "Np2ql_MBrkZtJ3cJDcR8dg",
            "alias": "rivers-and-roads-coffee-denver",
            "name": "Rivers and Roads Coffee",
            "image_url": "https://s3-media2.fl.yelpcdn.com/bphoto/Nha3gRx3hG9KkNLEoLva3A/o.jpg",
            "is_closed": false,
            "url": "https://www.yelp.com/biz/rivers-and-roads-coffee-denver?adjust_creative=-_Qqe9ZjDoBnVY8WtSd8og&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=-_Qqe9ZjDoBnVY8WtSd8og",
            "review_count": 167,
            "categories": [
                {
                    "alias": "coffee",
                    "title": "Coffee & Tea"
                },
                {
                    "alias": "breakfast_brunch",
                    "title": "Breakfast & Brunch"
                },
                {
                    "alias": "gluten_free",
                    "title": "Gluten-Free"
                }
            ],
            "rating": 4.5,
 

In [73]:
parsed_businesses = []
for payload in businesses_list:
    businesses = payload['businesses']
    for business in businesses:
        flattened_json = pd.json_normalize(business)
        flattened_json = flattened_json[['name','categories', 'location.display_address','display_phone']].to_dict()
        for k,v in flattened_json.items():
            if k == 'categories':
                categories = []
                for category in v[0]:
                    categories.append(category['title'])
                flattened_json[k] = categories
            elif k == 'location.display_address':
                flattened_json[k] =', '.join(v[0])
            else:
                flattened_json[k] = v[0]
            parsed_businesses.append(flattened_json)

TypeError: unhashable type: 'dict'

In [79]:
df = pd.DataFrame(parsed_businesses)
df = df.drop_duplicates(subset= ['name', 'location.display_address','display_phone'])
df = df.rename(columns={'location.display_address': 'address','display_phone': 'phone_number'})

In [81]:
df.to_csv('clayton_businesses_2021_02_21.csv')